In [ ]:
!python --version

In [2]:
!pip install openai


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# set openAI API key
import os
from openai import OpenAI
import pprint
import json
import math
client = OpenAI()

In [27]:
def gptSee(chatConfig, toolMap=None):
    # Return new messages
    new_messages = []
    
    # Required parameters
    messages = chatConfig.get('messages')
    model = chatConfig.get('model')
    if messages is None or model is None:
        raise ValueError("The 'messages' and 'model' parameters are required.")
    

    response = client.chat.completions.create(**chatConfig)
    # print()
    # print("\nResponse: \n" + pprint.pformat(response.model_dump()))
    if (response.choices[0].message.content):
        print("\nAssistant: \n" + response.choices[0].message.content)
   
    # append the messages but dump the None value in it.
    new_messages.append({k: v for k, v in response.choices[0].message.model_dump().items() if v is not None})
        
    return new_messages


In [28]:
messages = [
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://99designs-blog.imgix.net/blog/wp-content/uploads/2016/03/web-images.jpg?auto=format&q=60&w=1600&h=824&fit=crop&crop=faces"
          },
        },
      ],
    }    
]
model="gpt-4-vision-preview"

In [29]:
gptSee({"messages":messages, "model":model, "max_tokens": 4096})


Assistant: 
The image is a creative and whimsical digitally altered artwork showing a panda's head on a fish's body, swimming underwater among several other fish with similar panda-fish appearances. The artwork is designed to blend the distinctive black and white features of a panda bear with the bodies of fish, creating a fantasy-like hybrid animal. The scene is set underwater, with a slightly murky greenish-blue backdrop suggesting depth in the ocean environment.


[{'content': "The image is a creative and whimsical digitally altered artwork showing a panda's head on a fish's body, swimming underwater among several other fish with similar panda-fish appearances. The artwork is designed to blend the distinctive black and white features of a panda bear with the bodies of fish, creating a fantasy-like hybrid animal. The scene is set underwater, with a slightly murky greenish-blue backdrop suggesting depth in the ocean environment.",
  'role': 'assistant'}]

In [23]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "report_to_user",
      "description": "Report to user what animal is in the picture",
      "parameters": {
        "type": "object",
        "properties": {
          "animal": {
            "type": "string",
            "description": "The animal in the picture, e.g. Pig",
          }
        },
        "required": ["animal"],
      }
    }
  }
]

messages = [
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image? If this image has an animal, report the user what animal is in the pic by function calling"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://99designs-blog.imgix.net/blog/wp-content/uploads/2016/03/web-images.jpg?auto=format&q=60&w=1600&h=824&fit=crop&crop=faces"
          },
        },
      ],
    }    
]
model="gpt-4-vision-preview"
def report_to_user(animal):
    return {'report_success': True}

# Currently GPT4Vision does not support tools/functions